In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score, classification_report
from tkinter import Tk, Label, Entry, Button, StringVar, Frame
from tkinter.ttk import Combobox
import csv
import os

file_path = 'cardio_train.csv'
data = pd.read_csv(file_path, delimiter=';')


X = data.drop(columns=['cardio', 'id'])
y = data['cardio']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


logreg = LogisticRegression(max_iter=1000)
rfe = RFE(logreg, n_features_to_select=10)  # เลือกจำนวนฟีเจอร์ที่ต้องการ
X_train_rfe = rfe.fit_transform(X_train_scaled, y_train)
X_test_rfe = rfe.transform(X_test_scaled)


param_grid = {
    'C': [0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga']
}

grid_search = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=5)
grid_search.fit(X_train_rfe, y_train)


best_model = grid_search.best_estimator_

def predict_cardio(input_data):

    input_df = pd.DataFrame([input_data], columns=X.columns)

    input_scaled = scaler.transform(input_df)
    # RFE
    input_rfe = rfe.transform(input_scaled)
    # Predict
    prediction = best_model.predict(input_rfe)
    return prediction[0]

# ฟังก์ชันสำหรับการบันทึกผลการทำนาย
def save_prediction(input_data, prediction):
    file_exists = os.path.isfile('predictions.csv')
    with open('predictions.csv', mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=list(input_data.keys()) + ['prediction'])
        if not file_exists:
            writer.writeheader()
        input_data['prediction'] = prediction
        writer.writerow(input_data)

# สร้าง GUI
class CardioPredictor:
    def __init__(self, root):
        self.root = root
        self.root.title("Cardiovascular Disease Predictor")

        form_frame = Frame(root)
        form_frame.pack(pady=10)

     
        self.labels = []
        self.entries = []
        self.features = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
        for i, feature in enumerate(self.features):
            label = Label(form_frame, text=feature)
            label.grid(row=i, column=0, padx=5, pady=5, sticky='e')
            self.labels.append(label)

            entry = Entry(form_frame)
            entry.grid(row=i, column=1, padx=5, pady=5)
            self.entries.append(entry)

  
        self.combobox_features = ['gender', 'cholesterol', 'gluc', 'smoke', 'alco', 'active']
        self.comboboxes = []
        self.combobox_values = {
            'gender': ['1', '2'],
            'cholesterol': ['1', '2', '3'],
            'gluc': ['1', '2', '3'],
            'smoke': ['0', '1'],
            'alco': ['0', '1'],
            'active': ['0', '1']
        }
        start_row = len(self.features)
        for i, feature in enumerate(self.combobox_features):
            label = Label(form_frame, text=feature)
            label.grid(row=start_row + i, column=0, padx=5, pady=5, sticky='e')
            self.labels.append(label)

            combobox = Combobox(form_frame, values=self.combobox_values[feature])
            combobox.grid(row=start_row + i, column=1, padx=5, pady=5)
            self.comboboxes.append(combobox)


        action_frame = Frame(root)
        action_frame.pack(pady=10)

        self.result_var = StringVar()
        self.result_label = Label(action_frame, textvariable=self.result_var)
        self.result_label.grid(row=0, columnspan=2, pady=10)

        self.predict_button = Button(action_frame, text="Predict", command=self.predict)
        self.predict_button.grid(row=1, column=0, padx=10)

        self.clear_button = Button(action_frame, text="Clear", command=self.clear)
        self.clear_button.grid(row=1, column=1, padx=10)

        self.exit_button = Button(action_frame, text="Exit", command=self.exit_program)
        self.exit_button.grid(row=2, columnspan=2, pady=10)

    def predict(self):
        input_data = {}
        for feature, entry in zip(self.features, self.entries):
            input_data[feature] = float(entry.get())

        for feature, combobox in zip(self.combobox_features, self.comboboxes):
            input_data[feature] = int(combobox.get())

        result = predict_cardio(input_data)
        self.result_var.set(f"Prediction: {result} (0: No Disease, 1: Disease)")
        save_prediction(input_data, result)

    def clear(self):
        for entry in self.entries:
            entry.delete(0, 'end')
        for combobox in self.comboboxes:
            combobox.set('')
        self.result_var.set("")

    def exit_program(self):
        self.root.destroy()

if __name__ == "__main__":
    root = Tk()
    app = CardioPredictor(root)
    root.mainloop()
